In [1]:
import pandas as pd
#from modules.lavenshtein import lavenshtein, get_laven
from Levenshtein import distance
import os
from multiprocessing import Pool
from tqdm import tqdm
from modules.rev_helper import rev_helper

In [9]:
# test_df = pd.read_csv("data/original/test_transactions.csv")
# display(test_df)
# print(test_df.index.values.tolist())

,documentid,paymentid,amount,date,vendor_name,vendor_address
0,00d0472780579,00p0878907338,7.80,2018-4-4,RESTTORAN WAN SHENG,"NO.2, JALAN TEMENGUNG 19/9, SEKSYEM 9, BANDAR ..."
1,00d0269258457,00p0953953549,24.10,2017-9-13,AIK HUAT HARDWARE ENTERRPISE (SETIA ALAM) SDN BHD,"NO. 17-G, JALAN SETIA INDAH (X) U13/X, SETIA A..."
2,00d0145054354,00p0128923472,7.00,2017-11-12,SANYU STATIONERY SHOP,"NO. 31G&33G, JALAN SEITA INDAH X ,U13/X 40170 ..."
3,00d0795468800,00p0105962640,2.50,2018-3-5,SNG KEE CHSERAS RESTAURANT,"69, JALANTEMENGGUNG 3/9 BANDAR MAHKOTA CHERAS ..."
4,00d0682248746,00p0398282542,27.90,2018-1-28,99 SPEED MART S/B,"LOT P.T. 2811, JALAN ANGSA, TAAMN BERKELEY 411..."
...,...,...,...,...,...,...
121,00d0413686396,00p0979468067,18.90,2018-11-5,GERBANG ALAF RESTAURAMTS SDN BHD,"LEVEL 6, BANGUNAAN TH, DAMANSARA UPTOWN3 NO.3,..."
122,00d0745331693,00p0257799448,7.45,2018-4-26,KEDAI PAPAN YEW CHUAN,"LOT 276 JALAN BANTING 43800 DENGKIL, SELAMGOR"
123,00d0978814485,00p0847177660,227.90,2018-11-5,RESTAURANT JIAWEI JIAWEI HOUSE,"13, JLN TASIK UTAMA 8 MEDAN NIAGA DAMAI SG BES..."
124,00d0902251681,00p0637100547,7.50,2018-6-27,SANJUNG RALITI SDM. BHD.,"CAR PARK OFFICE, BASEMENT 1, LABUHRAYA SPRINT,..."


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125]


In [2]:
'''
iterates through all the ocr files an finds the Levenshtein distance between the file's 
text content and the provided user data.
outputs a dataframe with the documentid, a list of potential userids, and a list of their
corresponding Levenshtein distances
'''
columns = ["BB1", "BB2", "BB3", "BB4", "BB5", "BB6", "BB7", "BB8", "Text_Main"]
columns_labels = ["documentid","pot_doc","pot_name","pot_price","pot_date","pot_address",]
ocrdir = 'data/interim/ocr/'
levan_user = pd.DataFrame(columns=columns_labels)
#reads user data
users_data = pd.read_csv("data/interim/Users.csv")
#removes all files that are not present in the user file
file_names = [os.path.splitext(file)[0] for file in list(os.listdir(ocrdir))]
file_names = [x for x in file_names if x in users_data['documentid'].values]
file_names = [file_name + '.csv' for file_name in file_names]
#pools the function so that it runs in paralell
with Pool() as p:
    q = p.map(rev_helper, zip(file_names, [users_data for _ in range(len(os.listdir(ocrdir)))]))
for qq in q:
    levan_user.loc[len(levan_user)] = qq

In [3]:
df = levan_user

def remove_element_at_position(value):
    if not value['position']:
        return pd.Series({'documentid':value['documentid'],'pot_doc':value['pot_doc'],'pot_name':value['pot_name'], 'pot_price':value['pot_price'],'pot_address':value['pot_address'],'pot_date':value['pot_date']})
    pot_doc = [value['pot_doc'][i] for i in value['position']]
    pot_name = [value['pot_name'][i] for i in value['position']]
    pot_price = [value['pot_price'][i] for i in value['position']]
    pot_address = [value['pot_address'][i] for i in value['position']]
    pot_date = [value['pot_date'][i] for i in value['position']]
    return pd.Series({'documentid':value['documentid'],'pot_doc':pot_doc,'pot_name':pot_name, 'pot_price':pot_price,'pot_address':pot_address,'pot_date':pot_date})
        

df['position'] = df['pot_date'].apply(lambda x: [i for i, x in enumerate(x) if x == 1])

df = df.apply(remove_element_at_position, axis=1)

display((df))
df.to_csv(os.path.join('data/final/', "final.csv"))


,documentid,pot_doc,pot_name,pot_price,pot_date,pot_address
0,00d0243046961,"[00d0760612254, 00d0889453684, 00d0552638268, ...","[9, 9, 9, 9, 9, 9, 9, 5, 9, 9, 9]","[75, 50, 100, 75, 100, 75, 50, 67, 100, 67, 57]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[93, 93, 90, 90, 93, 90, 93, 90, 93, 90, 93]"
1,00d0164592408,"[00d0414196980, 00d0614628355, 00d0891802782, ...","[2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2]","[67, 67, 100, 67, 86, 86, 57, 67, 50, 75, 100,...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[96, 96, 92, 96, 44, 92, 54, 96, 92, 45, 96, 96]"
2,00d0696518680,"[00d0316894686, 00d0696518680, 00d0735262766, ...","[14, 1, 14, 1, 1]","[75, 100, 67, 67, 50]","[0, 1, 0, 0, 0]","[90, 97, 97, 50, 47]"
3,00d0530724847,"[00d0414196980, 00d0614628355, 00d0891802782, ...","[2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2]","[67, 67, 100, 67, 86, 86, 57, 67, 50, 75, 100,...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]","[96, 96, 92, 96, 44, 92, 54, 96, 92, 45, 96, 96]"
4,00d0770154989,"[00d0770154989, 00d0289417092]","[14, 14]","[75, 80]","[1, 0]","[93, 97]"
...,...,...,...,...,...,...
494,00d0844880238,"[00d0841246769, 00d0333208815, 00d0821795963, ...","[3, 3, 3, 3, 3, 13, 16, 0, 3, 0, 0, 3, 3, 3, 0...","[67, 67, 100, 100, 50, 67, 67, 75, 75, 67, 67,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[90, 93, 93, 90, 93, 93, 93, 93, 93, 93, 93, 9..."
495,00d0354051709,[00d0354051709],[12],[100],[1],[95]
496,00d0726523245,"[00d0624439175, 00d0726523245, 00d0447521528, ...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 12, 8, 8, 10...","[57, 100, 67, 100, 57, 67, 67, 75, 57, 100, 57...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[97, 93, 97, 97, 93, 97, 97, 97, 97, 97, 97, 9..."
497,00d0318453525,"[00d0318453525, 00d0694169739]","[14, 14]","[100, 100]","[1, 0]","[100, 100]"


hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi


,documentid,pot_doc,pot_name,pot_price,pot_address,pot_date
0,00d0243046961,[00d0243046961],[9],[100],[93],[1]
1,00d0164592408,[00d0164592408],[2],[86],[92],[1]
2,00d0696518680,[00d0696518680],[1],[100],[97],[1]
3,00d0530724847,"[00d0191077139, 00d0530724847]","[0, 2]","[50, 100]","[92, 96]","[1, 1]"
4,00d0770154989,[00d0770154989],[14],[75],[93],[1]
...,...,...,...,...,...,...
494,00d0844880238,"[00d0841246769, 00d0333208815, 00d0821795963, ...","[3, 3, 3, 3, 3, 13, 16, 0, 3, 0, 0, 3, 3, 3, 0...","[67, 67, 100, 100, 50, 67, 67, 75, 75, 67, 67,...","[90, 93, 93, 90, 93, 93, 93, 93, 93, 93, 93, 9...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
495,00d0354051709,[00d0354051709],[12],[100],[95],[1]
496,00d0726523245,[00d0726523245],[8],[100],[93],[1]
497,00d0318453525,[00d0318453525],[14],[100],[100],[1]
